<a href="https://colab.research.google.com/github/Nishimura35/weatherwise-marina-nishimura/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fetch-my-weather
!pip install hands-on-ai

import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

# Setup and Configuation
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response
import re

# =====================================
# 1. Weather Data Functions
# =====================================
# Get weather data
def get_weather_data(location: str, forecast_days: int = 3) -> dict:
  try:
    if forecast_days < 1 or forecast_days > 5:
      return {"error": "forecast_days must be between 1 and 5"}

    # Convert forecast_days to view_options (0 = 1 day, 1 = 2 days, ..., 4 = 5 days)
    view_option = str(forecast_days - 1)

    # Fetch raw text weather data
    raw_text = fetch_my_weather.get_weather(
      location=location,
      view_option=view_option,
      lang="en"
    )

    if isinstance(raw_text, str) and raw_text.startswith("Error:"):
      return {"error": raw_text}

    # Regex to extract high/low temps and precipitation
    temp_rain_pattern = r'([+\-]?\d+°C)\s*/\s*([+\-]?\d+°C).*?rain:\s*([\d.]+)mm'
    matches = re.findall(temp_rain_pattern, raw_text)

    # Regex to extract weather conditions (e.g., Sunny, Cloudy, Rain)
    condition_pattern = r'\n\s*[☀️🌤️🌧️🌩️❄️☁️⛅️]*\s*([A-Z][a-z]+)'
    conditions = re.findall(condition_pattern, raw_text)

    forecast = []
    for i in range(min(forecast_days, len(matches))):
      high, low, rain = matches[i]
      condition = conditions[i] if i < len(conditions) else "Unknown"
      forecast.append({
        "day": i + 1,
        "condition": condition,
        "high_temp": high,
        "low_temp": low,
        "precipitation_mm": float(rain)
        })

    return {
      "location": location,
      "forecast_days": len(forecast),
      "forecast": forecast
    }

  # Connection error
  except ConnectionError:
    print("Network error. Please check your connection and try again.")
    return None

  # Value error
  except ValueError as ve:
    print(f"Invalid input value: {ve}")
    return None

  # Unexpected error
  except Exception as e:
    print(f"An unexpected error occurred: {e}")
    return None

# =====================================
# 2. preprocess_forecast_data()
# =====================================
# Preprocess the forecast
def preprocess_forecast_data(raw_data):
  processed = []
  for entry in raw_data:
    high = int(entry['high_temp'].replace("°C", "").replace("+", ""))
    low = int(entry['low_temp'].replace("°C", "").replace("+", ""))
    processed.append({
      "day": entry['day'],
      "high_temp": high,
      "low_temp": low,
      "precipitation_mm": entry['precipitation_mm']
    })
  return processed

# =====================================
# 3. Visualisation Functions
# =====================================
# Show the weather forecast
def create_visualisation(forecast_data, location, output_type='display'):
  days = [entry['day'] for entry in weather_data]
  high_temps = [entry['high_temp'] for entry in weather_data]
  low_temps = [entry['low_temp'] for entry in weather_data]
  precipitation = [entry['precipitation_mm'] for entry in weather_data]

  fig, ax1 = plt.subplots(figsize=(10, 6))

  # Plot temperature lines on left y-axis
  ax1.plot(days, high_temps, color='red', marker='o', label='High Temp (°C)')
  ax1.plot(days, low_temps, color='blue', marker='o', label='Low Temp (°C)')
  ax1.set_xlabel('Day')
  ax1.set_ylabel('Temperature (°C)')
  ax1.legend(loc='upper left')

  # Create second y-axis for precipitation bars
  ax2 = ax1.twinx()
  ax2.bar(days, precipitation, alpha=0.3, color='blue', label='Precipitation (mm)')
  ax2.set_ylabel('Precipitation (mm)')
  ax2.legend(loc='upper right')

  for i, condition in enumerate([entry.get('condition', '') for entry in weather_data]):
    ax1.annotate(
      condition,
      (days[i], high_temps[i] + 1),
      textcoords="offset points",
      xytext=(0, 5),
      ha='center',
      fontsize=8,
      color='darkgreen'
    )

  plt.title(f'Weather Forecast for {location}: Temperature and Precipitation')
  plt.tight_layout()

  if output_type == 'figure':
    return fig
  elif output_type == 'save':
    plt.savefig(f"{weather_data['location']}_forecast.png")
    print("Graph saved!")
  else:
    plt.show()
    plt.close()


# Natural Language Processing
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    pass


def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

# =====================================
# 4. Main function
# =====================================
def main():
  print("Hi I'm Weatherwise App!")
  print("I can help you check the weather forecast.")

  while True:
    try:
      location = input("Which city are you interested in? (type 'exit' to quit)\n")
      if location.lower() == 'exit':
        print("Goodbye! Have a nice day!")
        break

      if not location:
        print("Please enter a city name.")
        continue

      weather_data = get_weather_data(location)
      if "error" not in weather_data:
        forecast_raw = weather_data['forecast']
        forecast_processed = preprocess_forecast_data(forecast_raw)

        print("\nWeather Forecast:")
        for raw, proc in zip(forecast_raw, forecast_processed):
          print(f"Day {proc['day']}: {raw['condition']}, "
          f"High: {proc['high_temp']}°C, Low: {proc['low_temp']}°C, "
          f"Rain: {proc['precipitation_mm']} mm")
          create_visualisation(forecast_processed, weather_data["location"])
      else:
        print(f"Sorry, I couldn't find weather data for '{location}'. Please try another city.")
        continue

    except ConnectionError:
      print("It looks like there's a problem with your internet connection. Please check and try again.")
    except Exception as e:
      print(f"Oops! Something went wrong: {e}")
      print("Please try again or contact support if the issue persists.")


# =====================================
# 5. Main Script
# =====================================
if __name__ == '__main__':
    main()



# Testing and Examples
# Include sample input/output for each function